In [1]:
cd /content/drive/My Drive/sunday

/content/drive/My Drive/sunday


In [2]:
ls

kaggle.json            Sunkeras.ipynb    test_df.csv.zip
keras_sub.csv          SunPytorch.ipynb  train_df.csv
sample_submission.csv  test_df.csv       train_df.csv.zip


In [0]:
import pandas as pd
train = pd.read_csv("train_df.csv")
test = pd.read_csv("test_df.csv")
sample_submission = pd.read_csv("sample_submission.csv")

In [0]:
X = train.iloc[:, 1:].values / 255
y = train.iloc[:, 0].values
test = test.iloc[:, 1:].values / 255

In [0]:
from torch.utils.data import Dataset, DataLoader
import torch
class TrainDataset(Dataset):
    def __init__(self, X, y, transform=None):
        self.X = X
        self.y = y
        self.transform = transform
    def __len__(self):
        return len(self.X)
    def __getitem__(self, idx):
        X, y = self.X[idx], self.y[idx]
        x2d = X.reshape(28, 28)
        x2d_aug = aug(x2d)
        x2d = X.reshape(784)
        return torch.tensor(X).float(), torch.tensor(y)

class TestDataset(Dataset):
    def __init__(self, X, y=None, transform=None):
        self.X = X
        self.y = y
        self.transform = transform
    def __len__(self):
        return len(self.X)
    def __getitem__(self, idx):
        X = self.X[idx]
        return torch.tensor(X).float()

In [0]:
traindataset = TrainDataset(X, y)
testdataset = TestDataset(test)

In [0]:
trainloader = DataLoader(traindataset, batch_size=256, shuffle=True, num_workers=2)
testloader = DataLoader(testdataset, batch_size=256, shuffle=False, num_workers=2)

In [0]:
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(28*28, 512)
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, 10)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

net = Net()

In [0]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=0.00001)

In [20]:
if torch.cuda.is_available():
    device = torch.device("cuda") 
print(device)   

cuda


In [0]:
net = net.to(device)

In [26]:
num_epochs = 20
for epoch in range(num_epochs):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader):
        # get the inputs; data is a list of [inputs, labels]
        
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
    print('[%d, %5d] loss: %.3f' %(epoch + 1, i + 1, running_loss / 20))
    running_loss = 0.0

print('Finished Training')

[1,   165] loss: 14.247
[2,   165] loss: 10.865
[3,   165] loss: 8.051
[4,   165] loss: 6.079
[5,   165] loss: 4.763
[6,   165] loss: 3.904
[7,   165] loss: 3.290
[8,   165] loss: 2.867
[9,   165] loss: 2.539
[10,   165] loss: 2.295
[11,   165] loss: 2.102
[12,   165] loss: 1.954
[13,   165] loss: 1.822
[14,   165] loss: 1.736
[15,   165] loss: 1.634
[16,   165] loss: 1.559
[17,   165] loss: 1.501
[18,   165] loss: 1.440
[19,   165] loss: 1.392
[20,   165] loss: 1.385
Finished Training


In [28]:
correct = 0
total = 0
preds = []
with torch.no_grad():
    for data in testloader:
        images = data
        images = images.to(device)
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        for pred in predicted:
          preds.append(pred.cpu().numpy())
print(preds[:10])

[array(8), array(0), array(5), array(3), array(8), array(1), array(9), array(6), array(6), array(0)]


In [30]:
sample_submission["Category"] = pd.Series(preds)
sample_submission.head()

,Id,Category
0,57808,8
1,4960,0
2,35755,5
3,15543,3
4,48968,8


In [0]:
sample_submission.to_csv("torch_sub.csv", index=False)